#📌 Extracão

🟦 1. Importar as bibliotecas necessárias

In [60]:
# Bibliotecas
import pandas as pd
import requests


🟦 2. Definir o link da API e fazer a requisição

In [61]:
# URL da API com os dados no formato JSON
url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/refs/heads/main/TelecomX_Data.json'

# Fazendo a requisição HTTP para obter os dados da API
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida (status code 200)
if response.status_code == 200:
    print("✅ Dados carregados com sucesso!")
else:
    print(f"❌ Falha ao carregar os dados. Código de status: {response.status_code}")


✅ Dados carregados com sucesso!


🟦 3. Converter os dados JSON em um DataFrame

In [62]:
# Convertendo o conteúdo da resposta em JSON
dados_json = response.json()

# Criando um DataFrame a partir dos dados JSON
df = pd.DataFrame(dados_json)

# Exibindo as 5 primeiras linhas do DataFrame para conferência
df.head()


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


---
As colunas customer, phone, internet e account estão armazenando dicionários (dict) como valores — ou seja, cada célula dessas colunas contém informações aninhadas que precisam ser "explodidas" (ou melhor, normalizadas ou expandida coluna a coluna).

---

🟦 4. Expandir (explodir) as colunas aninhadas

In [63]:
# Vamos fazer uma cópia do dataframe original, para não perdermos os dados iniciais
df_exp = df.copy()

# Explodindo cada coluna que contém um dicionário em novas colunas
# Usamos o método pd.json_normalize para "desempacotar" os dados de cada célula
# e depois concatenamos os resultados no dataframe original

# Coluna: customer
customer_df = pd.json_normalize(df_exp['customer'])
df_exp = pd.concat([df_exp.drop(columns=['customer']), customer_df], axis=1)

# Coluna: phone
phone_df = pd.json_normalize(df_exp['phone'])
df_exp = pd.concat([df_exp.drop(columns=['phone']), phone_df], axis=1)

# Coluna: internet
internet_df = pd.json_normalize(df_exp['internet'])
df_exp = pd.concat([df_exp.drop(columns=['internet']), internet_df], axis=1)

# Coluna: account
account_df = pd.json_normalize(df_exp['account'])
df_exp = pd.concat([df_exp.drop(columns=['account']), account_df], axis=1)

# Exibindo as primeiras linhas do novo DataFrame com colunas planificadas
df_exp.head()


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


🟦 5. Verificar os dados do dataframe

In [64]:
# Verificando as informações do dataframe
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


🟦 6. Salvar o DataFrame em um arquivo CSV

In [65]:
# Salvando o DataFrame em um arquivo CSV com separador padrão (vírgula)
# O arquivo será salvo no diretório raiz do Google Colab (pode ser baixado em seguida)
df_exp.to_csv('dados_telecomx.csv', index=False)

print("💾 Arquivo 'dados_telecomx.csv' salvo com sucesso!")


💾 Arquivo 'dados_telecomx.csv' salvo com sucesso!


#🔧 Transformação

🟦 1. Análise de valores ausentes em todas as colunas

In [66]:
# Importação das bibliotecas necessárias
import numpy as np

# Definição de valores que podem representar ausência (mesmo que não sejam NaN originalmente)
valores_ausentes = ["", " ", "None", "none", "NONE"]

# Substituímos esses valores por np.nan (o padrão para valores ausentes no Pandas)
df_analise = df_exp.replace(valores_ausentes, np.nan)

# Verificamos a quantidade de valores ausentes por coluna
valores_nulos_por_coluna = df_analise.isnull().sum()

# Exibimos o resultado: número de valores ausentes em cada coluna
print("Valores ausentes por coluna:")
print(valores_nulos_por_coluna)

# Se quisermos visualizar as linhas que têm pelo menos um valor ausente:
linhas_com_ausentes = df_analise[df_analise.isnull().any(axis=1)]

# Mostramos as primeiras 10 linhas com dados ausentes (para não poluir o notebook)
print("\nExemplo de linhas com valores ausentes:")
display(linhas_com_ausentes.head(10))

Valores ausentes por coluna:
customerID            0
Churn               224
gender                0
SeniorCitizen         0
Partner               0
Dependents            0
tenure                0
PhoneService          0
MultipleLines         0
InternetService       0
OnlineSecurity        0
OnlineBackup          0
DeviceProtection      0
TechSupport           0
StreamingTV           0
StreamingMovies       0
Contract              0
PaperlessBilling      0
PaymentMethod         0
Charges.Monthly       0
Charges.Total        11
dtype: int64

Exemplo de linhas com valores ausentes:


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
30,0047-ZHDTW,NaN,Female,0,No,No,11,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),79.00,929.3
75,0120-YZLQA,NaN,Male,0,No,No,71,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),19.90,1355.1
96,0154-QYHJU,NaN,Male,0,No,No,29,Yes,No,DSL,...,Yes,No,Yes,No,No,One year,Yes,Electronic check,58.75,1696.2
98,0162-RZGMZ,NaN,Female,1,No,No,5,Yes,No,DSL,...,Yes,No,Yes,No,No,Month-to-month,No,Credit card (automatic),59.90,287.85
175,0274-VVQOQ,NaN,Male,1,Yes,No,65,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,One year,Yes,Bank transfer (automatic),103.15,6792.45
219,0328-IBUPK,NaN,Female,0,Yes,Yes,51,No,No phone service,DSL,...,No,No,Yes,No,No,Two year,No,Bank transfer (automatic),34.20,1782
312,0448-YZNZE,NaN,Male,1,Yes,No,27,Yes,Yes,Fiber optic,...,Yes,Yes,No,No,Yes,Month-to-month,Yes,Electronic check,95.55,2510.2
351,0510-EXSMQ,NaN,Female,0,No,No,9,Yes,No,Fiber optic,...,No,No,No,No,No,Month-to-month,No,Electronic check,69.05,651.5
368,0530-HBKHZ,NaN,Female,1,Yes,No,1,No,No phone service,DSL,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,24.80,24.8
374,0534-JRNIG,NaN,Female,0,No,No,6,Yes,Yes,Fiber optic,...,No,No,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),93.55,536.4


---
Descobrimos que 224 linhas na coluna Churn possui ausência de valores, esta é a coluna mais importante no contexto da nossa análise, não podemos analisar aquilo que não se tem valor, minha decisão é excluir os dados com ausência de valores na coluna Churn.

---

🟦 1. Exclusão de linhas com valores ausentes na coluna Churn

In [67]:
# Remover linhas com Churn ausente (não podem ser usadas para modelagem)
df_analise = df_analise[~df_analise['Churn'].isnull()]

# Substituímos esses valores por np.nan (o padrão para valores ausentes no Pandas)
df_analise = df_analise.replace(valores_ausentes, np.nan)

# Verificamos a quantidade de valores ausentes por coluna
valores_nulos_por_coluna = df_analise.isnull().sum()

# Exibimos o resultado: número de valores ausentes em cada coluna
print("Valores ausentes por coluna:")
valores_nulos_por_coluna


Valores ausentes por coluna:


,0
customerID,0
Churn,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0


---
Temos 11 linhas da coluna Charges.Total com ausência de valores. Vamos separar estas 11 linhas para tentar descobrir o porquê.

---

In [68]:
# Substituindo valores ausentes representados por strings não convencionais
df_analise['Charges.Total'] = df_analise['Charges.Total'].replace(valores_ausentes, np.nan)

# Verificando as linhas onde Charges.Total está ausente
linhas_ausentes_charges = df_analise[df_analise['Charges.Total'].isnull()]

# Exibindo as linhas
print(f"Número de linhas com Charges.Total ausente: {len(linhas_ausentes_charges)}")
display(linhas_ausentes_charges)

Número de linhas com Charges.Total ausente: 11


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN
2308,3213-VVOLG,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN
2930,4075-WKNIU,No,Female,0,Yes,Yes,0,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN
3134,4367-NUYAO,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN
3203,4472-LVYGI,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN
4169,5709-LVOEQ,No,Female,0,Yes,Yes,0,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN


---

---
Percebemos que o motivo da coluna Charge.Total ter estas 11 linhas com ausência de valores é porque a coluna tenure está zerada, então toda vez que isto acontecer vamos preencher com 0. Vamos aproveitar também para transformar o dtype da coluna Charge.Total de object para flot64.

---

In [69]:
# Substituir Charges.Total ausente por 0 quando tenure == 0
df_analise['Charges.Total'] = df_analise['Charges.Total'].replace(["", " ", "None", "none", "NONE"], np.nan)
df_analise.loc[(df_analise['Charges.Total'].isnull()) & (df_analise['tenure'] == 0), 'Charges.Total'] = 0

# Converter Charges.Total para float
df_analise['Charges.Total'] = df_analise['Charges.Total'].astype(np.float64)

In [70]:
df_analise.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   Churn             7043 non-null   object 
 2   gender            7043 non-null   object 
 3   SeniorCitizen     7043 non-null   int64  
 4   Partner           7043 non-null   object 
 5   Dependents        7043 non-null   object 
 6   tenure            7043 non-null   int64  
 7   PhoneService      7043 non-null   object 
 8   MultipleLines     7043 non-null   object 
 9   InternetService   7043 non-null   object 
 10  OnlineSecurity    7043 non-null   object 
 11  OnlineBackup      7043 non-null   object 
 12  DeviceProtection  7043 non-null   object 
 13  TechSupport       7043 non-null   object 
 14  StreamingTV       7043 non-null   object 
 15  StreamingMovies   7043 non-null   object 
 16  Contract          7043 non-null   object 
 17  

---
Agora vamos verificar se há dados duplicados em nosso dataframe.

---

Verificação de Duplicatas com Base no customerID

In [71]:
# Verificando duplicatas com base apenas na coluna customerID
# keep=False marca todas as ocorrências duplicadas
duplicados_customerID = df_analise[df_analise.duplicated(subset='customerID', keep=False)]

# Exibindo a quantidade de customerID duplicados
qtd_duplicados = duplicados_customerID['customerID'].nunique()
print(f"Número de customerID duplicados: {qtd_duplicados}")

# Exibindo as linhas duplicadas, se houver
if qtd_duplicados > 0:
    display(duplicados_customerID)
else:
    print("✅ Nenhum customerID duplicado encontrado.")

Número de customerID duplicados: 0
✅ Nenhum customerID duplicado encontrado.


Verificação de Linhas Completamente Duplicadas

In [72]:
# Verificando se existem linhas completamente duplicadas no dataset
linhas_duplicadas = df_analise[df_analise.duplicated(keep=False)]

# Exibindo a quantidade total de duplicatas completas
print(f"Quantidade de linhas completamente duplicadas: {linhas_duplicadas.shape[0]}")

# Exibindo as linhas duplicadas, se houver
if not linhas_duplicadas.empty:
    display(linhas_duplicadas)
else:
    print("✅ Nenhuma linha completamente duplicada encontrada.")


Quantidade de linhas completamente duplicadas: 0
✅ Nenhuma linha completamente duplicada encontrada.


---
Vamos agora verificar possíveis inconsistências ou anomalias nos dados de cada coluna. Vamos listar os valores únicos existentes por coluna.

---

Verificação de Valores Únicos por Coluna

In [73]:
# Loop para verificar os valores únicos em cada coluna
for coluna in df_analise.columns:
    print(f"\n📌 Coluna: {coluna}")
    print(f"Tipo de dado: {df_analise[coluna].dtype}")
    print("Valores únicos:")
    print(df_analise[coluna].unique())
    print("-" * 50)


📌 Coluna: customerID
Tipo de dado: object
Valores únicos:
['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
--------------------------------------------------

📌 Coluna: Churn
Tipo de dado: object
Valores únicos:
['No' 'Yes']
--------------------------------------------------

📌 Coluna: gender
Tipo de dado: object
Valores únicos:
['Female' 'Male']
--------------------------------------------------

📌 Coluna: SeniorCitizen
Tipo de dado: int64
Valores únicos:
[0 1]
--------------------------------------------------

📌 Coluna: Partner
Tipo de dado: object
Valores únicos:
['Yes' 'No']
--------------------------------------------------

📌 Coluna: Dependents
Tipo de dado: object
Valores únicos:
['Yes' 'No']
--------------------------------------------------

📌 Coluna: tenure
Tipo de dado: int64
Valores únicos:
[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 37 49 66 67
 20 43 59 12 27  2 25 29 14 35 64 39 40 11  6 30 70 57 58 16 32 33 10 21
 6

---
Agora, com os dados limpos, é hora de criar a coluna "Contas_Diarias". Utilize o faturamento mensal para calcular o valor diário, proporcionando uma visão mais detalhada do comportamento dos clientes ao longo do tempo.

📌 Essa coluna ajudará a aprofundar sua análise e fornecer informações valiosas para as próximas etapas.

---

Criando coluna Contas.Diarias

In [75]:
# Criar a nova coluna Contas.Diarias dividindo Charges.Monthly por 30
df_analise['Contas.Diarias'] = df_analise['Charges.Monthly'] / 30

# Garantir que a nova coluna seja do tipo float64
df_analise['Contas.Diarias'] = df_analise['Contas.Diarias'].round(2).astype('float64')

df_analise.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Contas.Diarias
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.46
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,3.27
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,2.80


Gerando um novo arquivo .csv

In [78]:
df_analise.to_csv('dados_telecomx_transformados.csv', index=False)
print("💾 Arquivo 'dados_telecomx_transformados.csv' salvo com sucesso!")

df_dados_telecomx_transformados = df_analise

df_dados_telecomx_transformados.head()

💾 Arquivo 'dados_telecomx_transformados.csv' salvo com sucesso!


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Contas.Diarias
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.46
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,3.27
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,2.80


#📊 Carga e análise

#📄Relatorio Final